In [1]:
import torch
torch.backends.mps.is_available()  # 우왕 GPU를 쓸 수 있다...???

True

In [2]:
import os
path = '/Users/dayoungcho/Desktop/kaggle/statoil-iceberg-classifier-challenge'
os.chdir(path)

In [3]:
os.getcwd()

'/Users/dayoungcho/Desktop/kaggle/statoil-iceberg-classifier-challenge'

In [4]:
import numpy as np
import pandas as pd

from subprocess import check_output
print(check_output(['ls', path]).decode('utf8'))

sample_submission.csv
test.json
train.json



In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10,10
%matplotlib inline

/Users/dayoungcho/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
train = pd.read_json('train.json')
target_train = train['is_iceberg']
test = pd.read_json('test.json')

In [21]:
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce') # errors = 'coerce' -> 숫자로 변경할 수 없는 데이터라면 기존 데이터를 지우고 NaN으로 반환

train['inc_angle'] = train['inc_angle'].fillna(method='pad') # method='pad' -> 결측값이 위의 값과 동일하게 채워짐
test['inc_angle'] = test['inc_angle'].fillna(method='pad')

X_angle = train['inc_angle']
X_test_angle = test['inc_angle']

In [10]:
train.columns

Index(['id', 'band_1', 'band_2', 'inc_angle', 'is_iceberg'], dtype='object')

In [11]:
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_1']])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])
X_band_3 = (X_band_1 + X_band_2) / 2

X_train = np.concatenate([X_band_1[:,:,:,np.newaxis], X_band_2[:,:,:,np.newaxis], X_band_3[:,:,:,np.newaxis]], axis=-1)

In [12]:
X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_1']])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_2']])
X_band_test_3 = (X_band_test_1 + X_band_test_2) / 2

X_test = np.concatenate([X_band_test_1[:,:,:,np.newaxis], X_band_test_2[:,:,:,np.newaxis], X_band_test_3[:,:,:,np.newaxis]], axis=-1)

In [26]:
from matplotlib import pyplot
from keras import optimizers
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.models import Model
from keras import initializers
from keras.layers import LeakyReLU, PReLU
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

In [14]:
# data augmentation for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size = 64

# image transformation
gen = ImageDataGenerator(horizontal_flip = True, 
                         vertical_flip=True, 
                         width_shift_range = 0., 
                         height_shift_range = 0.,
                         channel_shift_range=0.,
                         zoom_range = 0.2,
                         rotation_range = 10
                         )

In [23]:
# function that merge two generators

def gen_flow_for_two_inputs(X1,X2,y):
    genX1 = gen.flow(X1,y,batch_size=batch_size, seed=55)
    genX2 = gen.flow(X1,X2,batch_size=batch_size, seed=55)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[1]], X1i[1]

# create generater

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only = True)
    return [es, msave]

def getVggAngleModel():
    input_2 = Input(shape=[1], name='angle')
    angle_layer = Dense(1,)(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x,angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [24]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)
        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])
        
    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [27]:
preds = myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


/Users/dayoungcho/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/var/folders/fd/1n9xc_8d2d51bf04p61cdylw0000gn/T/ipykernel_5715/2712130048.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  galaxyModel.fit_generator(


Epoch 1/100
24/24 [==============================] - 142s 6s/step - loss: 0.8224 - accuracy: 0.6256 - val_loss: 0.4644 - val_accuracy: 0.7234
Epoch 2/100
24/24 [==============================] - 157s 7s/step - loss: 0.4285 - accuracy: 0.7970 - val_loss: 0.2827 - val_accuracy: 0.8710
Epoch 3/100
24/24 [==============================] - 164s 7s/step - loss: 0.3408 - accuracy: 0.8451 - val_loss: 0.2822 - val_accuracy: 0.8710
Epoch 4/100
24/24 [==============================] - 155s 6s/step - loss: 0.2755 - accuracy: 0.8780 - val_loss: 0.2796 - val_accuracy: 0.8785
Epoch 5/100
21/24 [=========================>....] - ETA: 18s - loss: 0.2964 - accuracy: 0.8702

KeyboardInterrupt: 

In [ ]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)